In [2]:
import ipdb
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pylab as pylab
import cartopy
from utils import constants as cnst
from eod import msg
from utils import u_grid, u_interpolate as u_int, constants as cnst, u_arrays, u_darrays, u_met
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as feature
import cartopy.io.shapereader as shapereader
import pickle as pkl
import glob

coast = shapereader.natural_earth(resolution='110m',
                                  category='physical',
                                  name='coastline')

countries = shapereader.natural_earth(resolution='110m',
                                      category='cultural',
                                      name='admin_0_countries')

pylab.rcParams['figure.figsize'] = (50., 50.)
%matplotlib inline

In [1]:
cd ..

/home/ck/pythonWorkspace/proj_CEH


In [3]:
filetag = 'srfc'
era_hour = 12

In [12]:
def save_node_clim(filetag, node, era_hour):
    
    nodestr = 'node'+str(node)
    all_node_file = pkl.load(open(cnst.network_data + 'data/SOMS/saves/node_days.p', "rb"))
    node_days = all_node_file[nodestr]
    node_days_era = []
    for nd in node_days:
        node_days_era.append(nd.replace(hour=era_hour))
    node_days_era = pd.Series(node_days_era).values
    
    if filetag == 'pl':
        fft = 'pressure_levels'
    else:
        fft = 'surface'
        
    filebase = cnst.elements_drive + '/Africa/WestAfrica/ERA5/hourly/' 
    filepath = filebase + fft+'/ERA5*'+filetag+'.nc'
    out = '/home/ck/DIR/cornkle/data/SOMS/saves/'+nodestr+'_ERA5_clim_mean_'+filetag+'.nc'
    outdev = '/home/ck/DIR/cornkle/data/SOMS/saves/'+nodestr+'_ERA5_clim_stddev_'+filetag+'.nc'
    
    def means(da, node_days):
        
        da = da.sel(time=(da['time.hour']==era_hour))
        
        try:
            da = da.sel(level=[925,850,750,650,550,250])
        except:
            pass
        #ipdb.set_trace()
        intersect_dates = np.intersect1d(node_days, da.time.values)
        #ipdb.set_trace()
        if len(intersect_dates) == 0:
            return None
        try:
            da = da.sel(time=intersect_dates)
        except:
            return None

        da = u_darrays.flip_lat(da)
        return da
    
    srfc_list = []
    #ipdb.set_trace()
    for f in glob.glob(filepath):

        print('Doing', f)
        srfc = xr.open_dataset(f) 
        print('Opened', f)
#         if f == '/media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1991_04_pl.nc':
#             ipdb.set_trace()
        srfc_concat = means(srfc,node_days_era)
        if srfc_concat is None:
            print('SRFC Is NAN!!')
            continue
   
        srfc_list.append(srfc_concat)
    print('Concatenating node', node)
    #ipdb.set_trace()
    full = xr.concat(srfc_list, dim='time')
   
    mean = full.mean('time')
    dev = full.std('time')
    print('Starting to write file for node', node)
    mean.to_netcdf(out)
    dev.to_netcdf(outdev)

In [13]:
for ni in range(3,10):
#     nodestr='node'+str(1)
#     all_node_file = pkl.load(open(cnst.network_data + 'data/SOMS/saves/node_days.p', "rb"))
#     node_days = all_node_file[nodestr]
#     node_days_era = []
#     for nd in node_days:
#         node_days_era.append(nd.replace(hour=era_hour))
        
    save_node_clim(filetag, ni, era_hour)

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_01_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_01_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_02_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_02_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_03_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_03_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_04_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_04_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_05_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_05_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_0

Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1991_02_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1991_03_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1991_03_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1991_04_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1991_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1991_05_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1991_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1991_06_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1991_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1991_07_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1991_07_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/E

Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2009_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2010_06_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2010_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_05_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2012_12_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2012_12_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2015_07_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2015_07_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2017_11_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2017_11_srfc.nc
SRFC Is NAN!!
Doing /media/ck/E

SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_01_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_01_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_02_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_02_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_03_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_03_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_04_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2015_08_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2015_08_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2015_09_srfc.nc
Opened /media/ck/Elements//Afric

Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2001_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2001_08_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2001_08_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2001_09_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2001_09_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2001_10_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2001_10_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2001_11_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2001_11_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2001_12_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2001_12_srfc.nc
SRFC Is NAN!!
Doing /media/ck/E

...

Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2018_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2018_05_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2018_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2018_06_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2018_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2018_07_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2018_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2018_08_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2018_08_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2018_09_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2018_09_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/E

Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2009_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2009_08_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2009_08_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2009_09_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2009_09_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2009_10_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2009_10_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2009_11_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2009_11_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2009_12_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2009_12_srfc.nc
SRFC Is NAN!!
Doing /media/ck/E

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2004_10_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2004_10_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2004_11_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2004_11_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2004_12_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2004_12_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_06_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_07_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1998_08_srfc.nc
Opened /media/ck/E

Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2007_02_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2007_03_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2007_03_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2007_04_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2007_04_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2007_05_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2007_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2007_06_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2007_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2007_07_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2007_07_srfc.nc
Doing /media/ck/E

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_10_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_10_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_11_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_11_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_12_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_12_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2012_01_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2012_01_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2012_02_srfc.nc
Opened /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2012_02_srfc.nc
SRFC Is NAN!!
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2012_0